## Create a simple tensor with random items

In [1]:
import numpy as np

# Suppress scientific notation
np.set_printoptions(suppress=True)

# Generate randomly distributed parameters
params = np.random.uniform(low=-50, high=150, size=20)

# Make sure important values are at the beginning for better debugging
params[0] = params.max() + 1
params[1] = params.min() - 1
params[2] = 0

# Round each number to the second decimal place
params = np.round(params, 2)

# Print the parameters
print(params)

[132.93 -47.31   0.   -46.31  65.9    2.53   8.14 110.12 112.67 -20.71
  23.05  80.48  60.96  93.4  131.93   0.08 100.89  61.93 -32.74 -18.56]


## Define the quantization methods and quantize

In [2]:
def clamp(params_q: np.array, lower_bound: int, upper_bound: int) -> np.array:
    params_q[params_q < lower_bound] = lower_bound
    params_q[params_q > upper_bound] = upper_bound
    return params_q

def asymmetric_quantization(params: np.array, bits: int) -> tuple[np.array, float, int]:
    # Calculate the scale and zero point
    alpha = np.max(params)
    beta = np.min(params)
    scale = (alpha - beta) / (2**bits-1)
    zero = -1*np.round(beta / scale)
    lower_bound, upper_bound = 0, 2**bits-1
    # Quantize the parameters
    quantized = clamp(np.round(params / scale + zero), lower_bound, upper_bound).astype(np.int32)
    return quantized, scale, zero

def asymmetric_dequantize(params_q: np.array, scale: float, zero: int) -> np.array:
    return (params_q - zero) * scale

def symmetric_dequantize(params_q: np.array, scale: float) -> np.array:
    return params_q * scale

def symmetric_quantization(params: np.array, bits: int) -> tuple[np.array, float]:
    # Calculate the scale
    alpha = np.max(np.abs(params))
    scale = alpha / (2**(bits-1)-1)
    lower_bound = -2**(bits-1)
    upper_bound = 2**(bits-1)-1
    # Quantize the parameters
    quantized = clamp(np.round(params / scale), lower_bound, upper_bound).astype(np.int32)
    return quantized, scale

def quantization_error(params: np.array, params_q: np.array):
    # calculate the MSE
    return np.mean((params - params_q)**2)

(asymmetric_q, asymmetric_scale, asymmetric_zero) = asymmetric_quantization(params, 8)
(symmetric_q, symmetric_scale) = symmetric_quantization(params, 8)

print(f'Original:')
print(np.round(params, 2))
print('')
print(f'Asymmetric scale: {asymmetric_scale}, zero: {asymmetric_zero}')
print(asymmetric_q)
print('')
print(f'Symmetric scale: {symmetric_scale}')
print(symmetric_q)

Original:
[132.93 -47.31   0.   -46.31  65.9    2.53   8.14 110.12 112.67 -20.71
  23.05  80.48  60.96  93.4  131.93   0.08 100.89  61.93 -32.74 -18.56]

Asymmetric scale: 0.7068235294117647, zero: 67.0
[255   0  67   1 160  71  79 223 226  38 100 181 153 199 254  67 210 155
  21  41]

Symmetric scale: 1.046692913385827
[127 -45   0 -44  63   2   8 105 108 -20  22  77  58  89 126   0  96  59
 -31 -18]


In [3]:
# Dequantize the parameters back to 32 bits
params_deq_asymmetric = asymmetric_dequantize(asymmetric_q, asymmetric_scale, asymmetric_zero)
params_deq_symmetric = symmetric_dequantize(symmetric_q, symmetric_scale)

print(f'Original:')
print(np.round(params, 2))
print('')
print(f'Dequantize Asymmetric:')
print(np.round(params_deq_asymmetric,2))
print('')
print(f'Dequantize Symmetric:')
print(np.round(params_deq_symmetric, 2))

Original:
[132.93 -47.31   0.   -46.31  65.9    2.53   8.14 110.12 112.67 -20.71
  23.05  80.48  60.96  93.4  131.93   0.08 100.89  61.93 -32.74 -18.56]

Dequantize Asymmetric:
[132.88 -47.36   0.   -46.65  65.73   2.83   8.48 110.26 112.38 -20.5
  23.33  80.58  60.79  93.3  132.18   0.   101.08  62.2  -32.51 -18.38]

Dequantize Symmetric:
[132.93 -47.1    0.   -46.05  65.94   2.09   8.37 109.9  113.04 -20.93
  23.03  80.6   60.71  93.16 131.88   0.   100.48  61.75 -32.45 -18.84]


In [4]:
# Calculate the quantization error
print(f'{"Asymmetric error: ":>20}{np.round(quantization_error(params, params_deq_asymmetric), 2)}')
print(f'{"Symmetric error: ":>20}{np.round(quantization_error(params, params_deq_symmetric), 2)}')

  Asymmetric error: 0.04
   Symmetric error: 0.05
